# A protocol for movement data exploration

This notebook presents a systematic movement data exploration protocol. 


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
FIGSIZE = (600,400)
SMSIZE = 300
COLOR = 'darkblue'

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import datashader as ds
import holoviews as hv

from shapely.geometry import Point, LineString
from holoviews.operation.datashader import datashade, spread
from holoviews.element import tiles
from holoviews import opts, dim 

In [ ]:
input_files = [
    'E:/Geodata/AISDK/raw_ais/aisdk_20170701.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20170702.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20170703.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20170704.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20170705.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20170706.csv',
    'E:/Geodata/AISDK/raw_ais/aisdk_20180101.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20180102.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20180103.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20180104.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20180105.csv',
    #'E:/Geodata/AISDK/raw_ais/aisdk_20180106.csv'
]

In [ ]:
df = pd.read_csv(input_files[0], nrows=100)

In [ ]:
df.head()

In [ ]:
df['SOG'].hist(bins=100, figsize=(15,3))

In [ ]:
df = None
for input_file in input_files[:2]: 
    a = pd.read_csv(input_file, usecols=['# Timestamp', 'MMSI', 'Latitude', 'Longitude', 'SOG', 'Type of mobile', 'Ship type', 'Navigational status'])
    a = a[(a['Type of mobile'] == 'Class A') & (a.SOG>0)]
    a.drop(columns=['Type of mobile', 'SOG'], inplace=True)
    if df is None:
        df = a
    else:
        df = df.append(a)
    
df.rename(columns={'# Timestamp':'time', 'MMSI':'id', 'Latitude':'lat', 'Longitude':'lon', 'Ship type':'shiptype', 'Navigational status':'navstat'}, inplace=True)
df['time'] = pd.to_datetime(df['time'], format='%d/%m/%Y %H:%M:%S')

In [ ]:
df.loc[:, 'x'], df.loc[:, 'y'] = ds.utils.lnglat_to_meters(df.lon, df.lat)

df.set_index('time', inplace=True)

df['navstat'] = df['navstat'].astype('category')
df['shiptype'] = df['shiptype'].astype('category')

In [ ]:
df.head()

In [ ]:
print('Number of records: {} million'.format(round(len(df)/1000000)))

## A) Missing data

Checking for missing data is a common starting point for exploring new movement datasets. At this early stage, we usually start with raw location records that have yet to be aggregated into trajectories. Therefore, initial analyses look at elementary position records.

The following protocol steps target issues of missing data with respect to movement data's spatial, temporal, and attribute dimensions.


### A-1) Spatial gaps & outliers

To gain an overview, the analysis should start from the whole time span before drilling down. Spatial context (usually in the form of base maps) is essential when assessing spatial extent and gaps because context influences movement.

#### Spatial spread / extent & outliers (whole territory / all movers / whole time span)

This step addresses the question if the dataset covers the expected spatial extent. This can be as simple as checking the minimum and maximum coordinate values of the raw records. However, it is not uncommon to encounter spurious location records or outliers that are not representative of the actual covered extent. These outliers may be truly erroneous positions but can also be correct positions that happen to be located outside the usual extent. Looking at elementary position records only, it is usually not possible to distinguish these two cases. It is therefore necessary to take note of these outliers and investigate further in later steps.

TODO: consequences 

Classic scatter plots (or point maps) are helpful at this step. Point density maps (often called heat maps) on their default settings tend to hide outliers and are therefore not recommended.

In [ ]:
print(f'Spatial extent: x_min={df.lon.min()}, x_max={df.lon.max()}, y_min={df.lat.min()}, y_max={df.lat.max()}')

In [ ]:
def plot_basic_scatter(df, color='darkblue', title='', width=FIGSIZE[0], height=FIGSIZE[1], size=2):
    opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))
    pts = df.hvplot.scatter(x='x', y='y', datashade=True, cmap=[color, color], frame_width=width, frame_height=height, title=str(title))
    return tiles.OSM() * spread(pts, px=size)

In [ ]:
plot_basic_scatter(df)

In [ ]:
df = df[(df.lon>-90) & (df.lon<90) & (df.lat>0) & (df.lat<80)]

In [ ]:
cropped_df = df[(df.lon>0) & (df.lon<20) & (df.lat>52) & (df.lat<60)]
cropped_df['navstat'] = cropped_df['navstat'].astype('category')
cropped_df['shiptype'] = cropped_df['shiptype'].astype('category')
plot_basic_scatter(cropped_df)

#### Spatial gaps (selected areas / all movers / whole time span)

This step addresses the question if there are spatial gaps in the data coverage. Depending on the type of movers, gaps in certain spatial contexts are to be expected. For example, we wouldn't expect taxi locations in lakes. Other gaps may indicate issues with the data collection process or the data export used to generate the analysis dataset. Therefore, it is essential to evaluate these gaps in their spatial context using base maps showing relevant geographic features, such as the road network for vehicle data or navigation markers for vessel data. The visualization scale influences which size of gaps can be discovered. However, there are of course practical limitations to exploring ever more detailed scales and resulting continuously growing numbers of gaps.

TODO: consequences

Point density maps are helpful since they make it easy to identify areas with low densities, ignoring occasional outliers.

In [ ]:
def plot_point_density(df, title='', width=FIGSIZE[0], height=FIGSIZE[1]):
    opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))
    pts = df.hvplot.scatter(x='x', y='y', title=str(title), datashade=True, frame_width=width, frame_height=height)
    return tiles.OSM() * pts

In [ ]:
plot_point_density(df)

### A-2) Temporal gaps & outliers

#### Temporal extent & outliers (whole territory / all movers / whole time span)

This step addresses the question if the dataset covers the expected temporal extent. Similar to exploring the spatial extent, the obvious step is to determine the minimum and maximum timestamps first. Since GPS tracking requires accurate clocks to function, time information on the tracker is usually reliable. However, it is not guaranteed that these timestamps make it through the whole data collection and (pre)processing chain leading up to the exploratory analysis. For example, in some cases, tracker (or sender) time is replaced by receiver or storage time. Thus clock errors on the receiving or storage devices can result in unexpected timestamps.

TODO: consequences

Temporal charts, particularly record counts over time, are helpful to gain a first impression of the overall temporal extent and whether it is continuous or split into multiple time frames with little or no data in between.

In [ ]:
print(f'Temporal extent: {df.index.min()} to {df.index.max()}')

In [ ]:
TIME_SAMPLE = '15min'

df['id'].resample(TIME_SAMPLE).count()\
    .hvplot(title=f'Number of records per {TIME_SAMPLE}', width=FIGSIZE[0])

#### Temporal gaps in linear sequence & temporal cycles (whole territory / all movers / time spans)

This step addresses the question if there are temporal gaps in the dataset. Temporal gaps can be due to scheduled breaks in data collection, deliberate choices during data export, as well as unintended issues during data collection or (pre)processing. Similar to exploring spatial gaps, the temporal scale influences which size of gaps can be discovered. Temporal gaps can be one-time events or exhibit reoccurring patterns. For example, daily and weekly cycles are typical for human movement data.

TODO: consequences

Two-dimensional time histograms are helpful at this step.

In [ ]:
counts_df = df['id'].groupby([df.index.hour, pd.Grouper(freq='d')]).count().to_frame(name='n')
counts_df.rename_axis(['hour', 'day'], inplace=True)
counts_df.hvplot.heatmap(title='Record count', x='hour', y='day', C='n', width=FIGSIZE[0])

### A-3) Spatiotemporal changes / gaps

While the previous two steps looked at spatial gaps over the whole time span or temporal gaps for the whole territory, this step aims to explore spatiotemporal changes and gaps.

#### Changing extent

This step addresses the question whether there are changes in spatial extent over time. Changing spatial extent may be due to planned extensions or reductions of the data collection / observation area. Similarly, the extent is also expected to shift if the movers collectively change their location, as is the case, for example, with tracks of migrating birds.

TODO: consequences

Small multiples are helpful since they provide a quick way to compare extents during different time spans.

In [ ]:
def plot_multiple_by_day(df, day):
    return plot_basic_scatter(df[df.index.date==day], title=day, width=SMSIZE, height=SMSIZE)
    
def plot_multiples_by_day(df):
    days = df.index.to_period('D').unique()
    a = None
    for a_day in days:
        a_day = a_day.to_timestamp().date()
        plot = plot_multiple_by_day(df, a_day)
        if a is None: a = plot
        else: a = a  + plot
    return a

In [ ]:
plot_multiples_by_day(df).cols(2)

In [ ]:
plot_multiples_by_day(cropped_df).cols(2)

In [ ]:
def plot_multiple_by_hour_of_day(df, hour, fun):
    return fun(df[df.index.hour==hour], title=hour, width=SMSIZE, height=SMSIZE)
    
def plot_multiples_by_hour_of_day(df, hours=range(0,24), fun=plot_basic_scatter):
    a = None
    for hour in hours:
        plot = plot_multiple_by_hour_of_day(df, hour, fun)
        if a is None: a = plot
        else: a = a + plot
    return a

In [ ]:
#plot_multiples_by_hour_of_day(df[df.shiptype=='Fishing']).cols(2)
plot_multiples_by_hour_of_day(df, hours=[6,7,8,9]).cols(2)

#### Temporary gaps

This step addresses the question whether there are temporary gaps in the overall spatial coverage. Like temporary changes in the overall extent, temporary gaps can be due to mover behavior, as well as planned and unplanned changes of the data collection or (pre)processing workflows.

TODO: consequences

Small multiples of density maps or animated density maps are helpful at this step.

In [ ]:
plot_multiples_by_hour_of_day(cropped_df, hours=[6,7,8,9], fun=plot_point_density).cols(2)

### A-4) Attribute gaps

Some attributes may only be available during certain time spans / or in certain areas.

#### Spatial attribute gaps

This step addresses the question if there are areas with missing attribute data. Locally missing attribute data can be due to heterogeneous data collection system setups.

TODO: consequences

The methods used to explore spatial extent and gaps can be adopted to missing attribute data.

In [ ]:
CATEGORY = 'shiptype' #'navstat'
COLOR_HIGHLIGHT = 'red'
COLOR_BASE = 'grey'

cats = df[CATEGORY].unique()
#[cat for cat in cats]

In [ ]:
cmap = {} 
for cat in cats:
    cmap[cat] = COLOR_BASE
cmap['Unknown value'] = COLOR_HIGHLIGHT
cmap['Undefined'] = COLOR_HIGHLIGHT

In [ ]:
def plot_categorized_scatter(df, cat, title='', width=SMSIZE, height=SMSIZE, cmap=cmap):
    opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))
    pts = df.hvplot.scatter(x='x', y='y', datashade=True, by=cat, colormap=cmap, legend=True, frame_width=width, frame_height=height, title=str(title))
    return tiles.OSM() * pts

In [ ]:
tmp = df
unknown = tmp[(tmp[CATEGORY]=='Unknown value') | (tmp[CATEGORY]=='Undefined')]
known = tmp[(tmp[CATEGORY]!='Unknown value') & (tmp[CATEGORY]!='Undefined')]

( plot_categorized_scatter(tmp, CATEGORY, title='Categorized', width=SMSIZE, height=SMSIZE, cmap=cmap) + 
  plot_basic_scatter(unknown, COLOR_HIGHLIGHT, title='Unknown only', width=SMSIZE, height=SMSIZE, size=1) +
  plot_basic_scatter(known, COLOR_BASE, title='Known only', width=SMSIZE, height=SMSIZE, size=1)
)

#### Temporal attribute gaps

This step addresses the question if there are temporary gaps in attribute data. Changes to the data collection or (pre)processing workflow can affect which attributes are available during certain time spans.

TODO: consequences

The methods used to explore temporal extent and gaps can be adopted to missing attribute data.

In [ ]:
plot_multiples_by_day(unknown).cols(2)

### A-5) Gaps in trajectories

Depending on the method used for splitting tracks into trajectories, the resulting trajectories can include gaps. These gaps can be due to technical failure of the tracking device, the mover leaving the observable area, deliberate deactivation of the tracking device, or (pre)processing issues. 

TODO: consequences

TODO: method

In [ ]:
from math import sin, cos, atan2, radians, degrees, sqrt, pi

R_EARTH = 6371000  # radius of earth in meters
C_EARTH = 2 * R_EARTH * pi  # circumference

def compute_distance(row):
    lon1 = row['prev_lon']
    lat1 = row['prev_lat']
    lon2 = row['lon']
    lat2 = row['lat']
    delta_lat = radians(lat2 - lat1)
    delta_lon = radians(lon2 - lon1)
    a = sin(delta_lat/2) * sin(delta_lat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(delta_lon/2) * sin(delta_lon/2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    dist = R_EARTH * c
    return dist

def connect_pts(row):
    lon1 = row['prev_lon']
    lat1 = row['prev_lat']
    lon2 = row['lon']
    lat2 = row['lat']
    return LineString([(lon1, lat1), (lon2, lat2)])

def find_gaps(df, min_dist, max_dist):
    if len(df)<2:
        return None
    i = df.copy()
    i = i.assign(prev_lon=i.lon.shift())
    i = i.assign(prev_lat=i.lat.shift())
    i = i.assign(dist=i.apply(compute_distance, axis=1))
    i = i[(i.dist>min_dist) & (i.dist<max_dist)]
    if len(i)==0: 
        return None
    i = i.assign(geometry=i.apply(connect_pts, axis=1))
    return i

def make_gap_gdf(df, min_dist, max_dist):
    a = None
    for the_id in df.id.unique():
        i = df[df.id==the_id]
        gaps_df = find_gaps(i, min_dist, max_dist)
        if gaps_df is None:
            continue
        if a is None: 
            a = gaps_df
        else:
            a = a.append(gaps_df)
    if a is not None:
        return gpd.GeoDataFrame(a, geometry='geometry')

def plot_gaps(df, min_dist, max_dist, width=FIGSIZE[0], height=FIGSIZE[1]):
    gaps_gdf = make_gap_gdf(df, min_dist, max_dist)
    if gaps_gdf is not None:
        plot = gaps_gdf.hvplot(geo=True, color=COLOR_HIGHLIGHT, frame_width=width, frame_height=height)
        return tiles.OSM() * plot   

In [ ]:
plot_gaps(cropped_df, min_dist=10000, max_dist=100000)

# Appendix -- Experiments

In [ ]:
sample_df = df[(df['id']==304752000) | (df['id']==257024000)]
sample_df.head()

In [ ]:
for name, df in sample_df.groupby('id'):
    print(f'{name}: {len(df)}')

In [ ]:
grouped = [df[['id','x','y']] for name, df in sample_df.groupby(['id', pd.Grouper(freq='d')])]
path = hv.Path(grouped, kdims=['x','y'], vdims=['id','x']).opts(line_width=2, width=600, color=COLOR)
plot = datashade(path).opts(frame_height=FIGSIZE[1], frame_width=FIGSIZE[0])
tiles.OSM() * plot

In [ ]:
grouped = [df[['x','y']] for name, df in cropped_df.groupby(['id', pd.Grouper(freq='d')]) if len(df)>100]
path = hv.Path(grouped, kdims=['x','y'])
plot = datashade(path).opts(frame_height=FIGSIZE[1], frame_width=FIGSIZE[0])
tiles.OSM() * plot

In [ ]:
tmp = cropped_df

a = None
for the_id in tmp.id.unique():
    i = tmp[tmp.id==the_id].copy()
    i = i.assign(prev_lon=i.lon.shift())
    i = i.assign(prev_lat=i.lat.shift())
    i = i.assign(dist=i.apply(compute_distance, axis=1))    
    #i = find_gaps(i, 1000, 10000000)
    plot = hv.Path(i, kdims=['x','y'], vdims=['id', 'dist']).opts(color='dist', line_width=4)
    plot = datashade(plot, normalization='linear', aggregator=ds.by('id', ds.min("dist")))
    #plot = datashade(plot, normalization='linear')
    if a is None: a = plot
    else: a = a * plot
tiles.OSM() * a


In [ ]:
a * sample_df.hvplot.scatter(x='x', y='y', datashade=True, by='id', frame_width=FIGSIZE[0], frame_height=FIGSIZE[1])

In [ ]:
datashade(hv.Path(sample_df, kdims=['x','y']), normalization='linear', aggregator=ds.any())

In [ ]:
tmp = sample_df[['id','x','y']]
hv.Path(tmp[tmp.id==304752000], kdims=['x','y'])